## Repo statistics for trydotnet

In [1]:
#r "nuget:Octokit, 0.32.0"
#r "nuget:NodaTime, 2.4.6"
using Octokit;
using NodaTime;
using NodaTime.Extensions;
using XPlot.Plotly;

Installing package Octokit, version 0.32.0..............done!

Successfully added reference to package Octokit, version 0.32.0

Installing package NodaTime, version 2.4.6....done!

Successfully added reference to package NodaTime, version 2.4.6

## setup
Creating GitHub public api client

In [2]:
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

Create check points for today, start of current month and start of current year

In [3]:
var today = SystemClock.Instance.InUtc().GetCurrentDate();
var startOfTheMonth = today.With(DateAdjusters.StartOfMonth);
var startOfTheYear = new LocalDate(today.Year, 1, 1).AtMidnight();

Querying Github for
* issues crate this month
* issued closed this month
* all issues for this year

In [5]:
var createdIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                Filter = IssueFilter.Created
            };
var closedIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                State = ItemStateFilter.Closed
            };
var thisYearIssuesRequest = new RepositoryIssueRequest
            {
                State = ItemStateFilter.All,
                Since = startOfTheYear.ToDateTimeUnspecified()
            };

Pulling data via GitHub api

In [6]:
var createdThisMonth = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", createdIssuesRequest);
var closedThisMonth = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", closedIssuesRequest);
var thisYearIssues = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", thisYearIssuesRequest);

Grouping open and closed issues by month

In [7]:
var openSoFar = createdThisMonth.OrderBy(i => i.CreatedAt).Where(i => i.State.StringValue == "open").ToArray();
var openByMonthOfCreation = openSoFar.GroupBy(i => new { i.CreatedAt.Year, i.CreatedAt.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();

var closedSoFar = thisYearIssues.OrderBy(i => i.CreatedAt).Where(i => i.State.StringValue == "closed").ToArray();
var closedByMonthOfClosure = closedSoFar.GroupBy(i => new { i.ClosedAt.Value.Year, i.ClosedAt.Value.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();
var totalOpenIssues = thisYearIssues.Count();
var openCountByMonth = closedSoFar.GroupBy(i => new { i.CreatedAt.Year, i.CreatedAt.Month})
                .Select(g => {
                    var count = g.Count();                    
                    var dataPoint = new  {Date = g.Key, Count = totalOpenIssues};
                    totalOpenIssues -= count;
                    return dataPoint;
                }).ToArray();

Showing issues open this month grouped by day

In [8]:
var createdThisMonthByDay = createdThisMonth.GroupBy(i => new DateTime(i.CreatedAt.Year,i.CreatedAt.Month, i.CreatedAt.Day)); 
var lineChart = Chart.Line(createdThisMonthByDay.Select(g => new Tuple<DateTime,int>(g.Key, g.Count())));
lineChart.WithTitle("Issue Open By Day");

display(lineChart);

In [77]:
display(openSoFar.Select(i => new {i.CreatedAt, i.Title, State = i.State.StringValue,  i.Number}).OrderByDescending(d => d.CreatedAt));

index,CreatedAt,Title,State,Number
0,2019-10-16 22:52:01Z,Sample Code on page 2 of tutorial should be swapped with page 3 of tutorial,open,536
1,2019-10-13 10:55:55Z,static async Task Main(string[] args),open,527
2,2019-10-12 13:26:24Z,Unable to use extension methods in .NET Jupyter Kernel,open,525
3,2019-10-12 12:17:41Z,New C# 8 Features Unusable on Try.Dot.NET,open,524
4,2019-10-11 22:38:06Z,"We need to have display/preview function to show unstructured data like images, videos, sound",open,523
5,2019-10-11 22:17:03Z,"notebook kernel stop the process when it calls ""console.writeline"" method.",open,522
6,2019-10-09 22:01:03Z,terminal formatter and mime type support,open,516
7,2019-10-06 12:48:15Z,Focus mode does not work in Safari on iOS 13 or macOS Catalina,open,500
8,2019-10-01 19:23:33Z,#r directive parsing probably needs to be language specific,open,491
9,2019-08-14 21:03:49Z,Fix warnings,open,375


In [9]:
var lineChart = Chart.Line(openByMonthOfCreation.Select(g => new Tuple<DateTime, int>(new DateTime(g.Date.Year, g.Date.Month, 1),g.Count)));
lineChart.WithTitle("Issue still open grouped by age (month they were created)");
display(lineChart);

In [10]:
var idleByMonth = openSoFar.Where( i => i.Comments == 0).GroupBy(i => new DateTime( i.CreatedAt.Year, i.CreatedAt.Month, 1))
    .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();
var activeByMonth = openSoFar.Where( i => i.Comments > 0).GroupBy(i => new DateTime( i.CreatedAt.Year, i.CreatedAt.Month, 1))
                .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();

var idleSeries = new Graph.Scattergl
{
    name = "Idle",
    y = idleByMonth.Select(g => g.Count ).ToArray(),
    x = idleByMonth.Select(g => g.Date ).ToArray()
};

var activeSeries = new Graph.Scattergl
{
    name = "Active",
    y = activeByMonth.Select(g => g.Count ).ToArray(),
    x = activeByMonth.Select(g => g.Date ).ToArray()
};


var chart = Chart.Plot(new[] {idleSeries, activeSeries});
chart.WithTitle("Idle and active open issue report");
display(chart);

In [11]:
var openDataPoints = openByMonthOfCreation
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();


var closedDataPoints = closedByMonthOfClosure
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();

var openCountByMonthDataPoints = openCountByMonth
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();

var openSeries = new Graph.Scattergl
{
    name = "Created",
    y = openDataPoints.Select(g => g.y ).ToArray(),
    x = openDataPoints.Select(g => g.x ).ToArray()
};

var closeSeries = new Graph.Scattergl
{
    name = "Closed",
    y = closedDataPoints.Select(g => g.y ).ToArray(),
    x = closedDataPoints.Select(g => g.x ).ToArray()
};

var stillOpenSeries = new Graph.Scattergl
{
    name = "Open",
    y = openCountByMonthDataPoints.Select(g => g.y ).ToArray(),
    x = openCountByMonthDataPoints.Select(g => g.x ).ToArray()
};


var chart = Chart.Plot(new[] {openSeries, closeSeries, stillOpenSeries});
chart.WithTitle("Issue report for the current year");
display(chart);

In [12]:
var forks = await gitHubClient.Repository.Forks.GetAll("dotnet", "try");

In [13]:
var forkCreateByMonth = forks.GroupBy(f => new DateTime(f.CreatedAt.Year, f.CreatedAt.Month,  f.CreatedAt.Day) )
    .Select(g => new {Date = g.Key, Count = g.Count()}).OrderBy(g => g.Date).ToArray();
var forkLastUpdateByMonth = forks.GroupBy(f => new DateTime(f.UpdatedAt.Year, f.UpdatedAt.Month,  f.UpdatedAt.Day) )
    .Select(g => new {Date = g.Key, Count = g.Count()}).OrderBy(g => g.Date).ToArray();

var total = 0;
var forkCountByMonth = forkCreateByMonth.OrderBy(g => g.Date).Select(g => new {g.Date, Count = total += g.Count}).ToArray();

var forkCreationSeries = new Graph.Scattergl
{
    name = "created by month",
    y = forkCreateByMonth.Select(g => g.Count ).ToArray(),
    x = forkCreateByMonth.Select(g => g.Date ).ToArray()
};

var forkTotalSeries = new Graph.Scattergl
{
    name = "running total",
    y = forkCountByMonth.Select(g => g.Count ).ToArray(),
    x = forkCountByMonth.Select(g => g.Date ).ToArray()
};

var forkUpdateSeries = new Graph.Scattergl
{
    name = "last update by month",
    y = forkLastUpdateByMonth.Select(g => g.Count ).ToArray(),
    x = forkLastUpdateByMonth.Select(g => g.Date ).ToArray()
};



var chart = Chart.Plot(new[] {forkCreationSeries,forkTotalSeries,forkUpdateSeries});
chart.WithTitle("Fork activity");
display(chart);